# Guided Project: Answering Business Questions using SQL
- **About:** 
    - We will be working with a modified version of the [Chinook](https://github.com/lerocha/chinook-database) database to answer business questions. The Chinook database contains information about a fictional digital music shop.
    - We will use python to work with a SQLite database.
- **Questions:** 
    - Which genres sell the best in the USA.
    - How each employee is performing in terms of sales.
    - What is the volume of sales per country.
    - Which way customers buy more, whole albums or individual tracks.

In [1]:
#############################################
# Defining utility functions to work with SQL
#############################################

import sqlite3
import pandas as pd

def run_query(q):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(q, conn)

def run_command(c):
    with sqlite3.connect('chinook.db') as conn:
        conn.isolation_level = None
        conn.execute(c)

def show_tables():
    q = 'SELECT name, type FROM sqlite_master WHERE type IN ("table","view");'
    return run_query(q)

show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


## Selecting Albums to Purchase

* Task: Select 3 albums from a list of 4 to be added to the Chinook store.
* Selection is based on what genres sell the best in the USA.

| Artist Name          | Genre   |
|----------------------|---------|
| Regal                | Hip-Hop |
| Red Tone             | Punk    |
| Meteor and the Girls | Pop     |
| Slim Jim Bites       | Blues   |

In [2]:
best_selling_genres_query = '''
WITH invoices_USA AS (
      SELECT il.invoice_line_id, il.track_id
      FROM invoice_line il
      INNER JOIN invoice i ON i.invoice_id = il.invoice_id
      INNER JOIN customer c ON c.customer_id = i.customer_id
      WHERE c.country = "USA"
)

SELECT g.name genre, COUNT(iu.invoice_line_id) tracks_sold,
       CAST(COUNT(iu.invoice_line_id) AS FLOAT) / (
            SELECT COUNT(*) FROM invoices_USA) AS tracks_sold_percent
FROM invoices_USA iu
INNER JOIN track t ON t.track_id = iu.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY genre ORDER BY tracks_sold DESC
'''
run_query(best_selling_genres_query)

,genre,tracks_sold,tracks_sold_percent
0,Rock,561,0.533777
1,Alternative & Punk,130,0.123692
2,Metal,124,0.117983
3,R&B/Soul,53,0.050428
4,Blues,36,0.034253
5,Alternative,35,0.033302
6,Latin,22,0.020932
7,Pop,22,0.020932
8,Hip Hop/Rap,20,0.019029
9,Jazz,14,0.013321


From the 4 possible albums, the genres that sell the most in the USA are Punk, Blues and Pop.
Choice of artists: Red Tone, Slim Jim Bites and Meteor and the Girls.

## Analyzing Employee Sales Performance

* Task: analyze employee performance, in terms of the total purchases of customers belonging to each employee.

In [3]:
employee_perform_query = '''
SELECT e.employee_id,
       e.first_name || " " || e.last_name employee_name,
       e.hire_date,
       SUM(i.total) total_sales
  FROM employee e
 INNER JOIN customer c ON c.support_rep_id = e.employee_id
 INNER JOIN invoice i ON i.customer_id = c.customer_id
 GROUP BY e.employee_id
 ORDER BY total_sales DESC
'''
run_query(employee_perform_query)

,employee_id,employee_name,hire_date,total_sales
0,3,Jane Peacock,2017-04-01 00:00:00,1731.51
1,4,Margaret Park,2017-05-03 00:00:00,1584.00
2,5,Steve Johnson,2017-10-17 00:00:00,1393.92


The best performing sales support agent is Jane Peacock, followed by Margaret Park and Steve Johnson.
We should take into account that some employees have working for the company for a longer period than others. The difference in the total sales may in part be explained by this rather than any difference in employee performance.

## Analyzing Sales by Country

* Task: for each country, calculate data related with sales performance. In particular:
    * total number of customers
    * total value of sales
    * average value of sales per customer
    * average order value

In [4]:
country_sales_query = '''
WITH total_value_customer AS (
        SELECT c.country,
               c.customer_id,
               SUM(i.total) total_value_customer,
               AVG(i.total) average_value_order_cust
        FROM customer c
        LEFT JOIN invoice i ON i.customer_id = c.customer_id
        GROUP BY c.country, c.customer_id
     ),
     statistics_country AS (
        SELECT tvc.country,
               COUNT(tvc.customer_id) total_cust,
               SUM(tvc.total_value_customer) total_value,
               AVG(tvc.total_value_customer) avg_value_cust,
               AVG(tvc.average_value_order_cust) avg_order_value
        FROM total_value_customer tvc
        GROUP BY tvc.country
     ),
     stat_country_others AS (
        SELECT sc.*, 0 AS sort
        FROM statistics_country sc
        WHERE sc.total_cust > 1
        
        UNION
        
        SELECT "Other" country,
               SUM(sc.total_cust) total_cust,
               SUM(sc.total_value) total_value,
               AVG(sc.avg_value_cust) avg_value_cust,
               AVG(sc.avg_order_value) avg_order_value,
               1 AS sort
        FROM statistics_country sc
        WHERE sc.total_cust <= 1
     )
SELECT country,
       total_cust,
       total_value,
       avg_value_cust,
       avg_order_value
FROM stat_country_others
ORDER BY sort, total_cust DESC
'''
run_query(country_sales_query)

,country,total_cust,total_value,avg_value_cust,avg_order_value
0,USA,13,1040.49,80.037692,8.014750
1,Canada,8,535.59,66.948750,7.416238
2,Brazil,5,427.68,85.536000,6.987485
3,France,5,389.07,77.814000,7.835000
4,Germany,4,334.62,83.655000,8.188750
5,United Kingdom,3,245.52,81.840000,8.749583
6,Czech Republic,2,273.24,136.620000,9.377500
7,India,2,183.15,91.575000,8.757692
8,Portugal,2,185.13,92.565000,6.527813
9,Other,15,1094.94,72.996000,7.445071


## Albums vs Individual Tracks
* Task: Understand wich way customers make more purchases:
    * purchasing a whole album.
    * purchasing a collection of one or more individual tracks.

In [5]:
album_tracks_query = '''
 
WITH is_album_or_not AS (
SELECT inv_list.invoice_id, 
       CASE WHEN 
            (
            SELECT il.track_id
            FROM invoice_line il
            WHERE il.invoice_id = inv_list.invoice_id
    
            EXCEPT 
    
            SELECT t.track_id
            FROM track t
            WHERE t.album_id = (SELECT t2.album_id
                                FROM track t2
                                INNER JOIN invoice_line il ON il.track_id = t2.track_id
                                WHERE il.invoice_id = inv_list.invoice_id
                                LIMIT 1
                                )
            ) IS NULL
            AND
            (
            SELECT t.track_id
            FROM track t
            WHERE t.album_id = (SELECT t2.album_id
                                FROM track t2
                                INNER JOIN invoice_line il ON il.track_id = t2.track_id
                                WHERE il.invoice_id = inv_list.invoice_id
                                LIMIT 1
                                )
            
            EXCEPT
            
            SELECT il.track_id
            FROM invoice_line il
            WHERE il.invoice_id = inv_list.invoice_id
            
            ) IS NULL
            
            THEN "album"
            ELSE "individual_tracks"
            END AS is_album
FROM invoice AS inv_list ) 

SELECT is_album type_of_invoice,
       COUNT(is_album) no_invoices,
       CAST(COUNT(is_album) AS FLOAT) / (SELECT COUNT(*) FROM invoice) percent_invoices
FROM is_album_or_not
GROUP BY is_album
'''
run_query(album_tracks_query)

,type_of_invoice,no_invoices,percent_invoices
0,album,114,0.185668
1,individual_tracks,500,0.814332
